# 分子データセットの作成とGCNの学習

このチュートリアルでは、`6_load_data`の拡張として、分子データセットを作成し、GCNで学習する。

事前に以下のライブラリをインストールする。dgllifeはライフサイエンス向けのdglライブラリで、化学・生物分野におけるグラフデータを扱うのに有用である。
```bash
pip install dgllife
pip install rdkit-pypi

In [1]:
try:
    from dgllife.utils import BaseAtomFeaturizer, atom_type_one_hot, atom_degree_one_hot, atom_total_num_H_one_hot, \
        atom_is_aromatic_one_hot, atom_hybridization_one_hot, atom_formal_charge_one_hot, atom_num_radical_electrons_one_hot, \
        bond_type_one_hot, bond_is_conjugated_one_hot, bond_is_in_ring_one_hot, bond_stereo_one_hot, BaseBondFeaturizer, ConcatFeaturizer
except:
    !pip install dgllife
    from dgllife.utils import BaseAtomFeaturizer, atom_type_one_hot, atom_degree_one_hot, atom_total_num_H_one_hot, \
        atom_is_aromatic_one_hot, atom_hybridization_one_hot, atom_formal_charge_one_hot, atom_num_radical_electrons_one_hot, \
        bond_type_one_hot, bond_is_conjugated_one_hot, bond_is_in_ring_one_hot, bond_stereo_one_hot, BaseBondFeaturizer, ConcatFeaturizer

try:
    from rdkit import Chem
except:
    !pip install rdkit-pypi
    from rdkit import Chem

import multiprocessing

from dgl.data import DGLDataset
from dgllife.utils import smiles_to_bigraph
import numpy as np
import pandas as pd
from rdkit import Chem
import torch

## Featurizerの実装

Featurizerの実装には、``dgllife.utils``に実装されている関数を使用する。各関数は、分子の特徴量を計算するための関数であるが、グラフのノードおよびエッジに特徴量として保持するために、各関数をリスト化したものを``ConcatFeaturizer``に入力する。これにより各関数の出力を連結したものを特徴量として出力することが可能である。

In [2]:
atom_feats_funcs = [atom_type_one_hot, 
                    atom_degree_one_hot,
                    atom_total_num_H_one_hot,
                    atom_is_aromatic_one_hot,
                    atom_hybridization_one_hot,
                    atom_formal_charge_one_hot,
                    atom_num_radical_electrons_one_hot,
                    ]

bond_feats_funcs = [bond_type_one_hot,
                    bond_is_conjugated_one_hot,
                    bond_is_in_ring_one_hot,
                    bond_stereo_one_hot,
                    ]

# ConcatFeaturizerにより、特徴量を連結する
atom_concat_featurizer = ConcatFeaturizer(atom_feats_funcs)
bond_concat_featurizer = ConcatFeaturizer(bond_feats_funcs)

# BaseAtomFeaturizer, BaseBondFeaturizerを用いて、特徴量を作成する
mol_atom_featurizer = BaseAtomFeaturizer({'h': atom_concat_featurizer})
mol_bond_featurizer = BaseBondFeaturizer({'e': bond_concat_featurizer})

サンプルとして、ベンゼン``'c1ccccc1'``で特徴量を計算する。原子特徴量は (6, 76), 結合特徴量は (12, 14) のテンソルとして出力される。

In [3]:
smi = 'c1ccccc1'
mol = Chem.MolFromSmiles(smi)
atom_feats_ex = mol_atom_featurizer(mol)
bond_feats_ex = mol_bond_featurizer(mol)
print("atom feature:", atom_feats_ex['h'].size())
print("bond feature:", bond_feats_ex['e'].size())

atom feature: torch.Size([6, 76])
bond feature: torch.Size([12, 14])


## SMILESからグラフの構築
smilesからグラフを構築するには、[```SMILESToBigraph```](https://lifesci.dgl.ai/generated/dgllife.utils.smiles_to_bigraph.html)を使用する。molオブジェクトから作成する場合、[```MolToBigraph```](https://lifesci.dgl.ai/generated/dgllife.utils.MolToBigraph.html)が使える。  
例としてエタノール``'CCO'``をグラフに変換する。

In [4]:
smi = 'CCO'
g = smiles_to_bigraph(smi, node_featurizer=mol_atom_featurizer,
                      edge_featurizer=mol_bond_featurizer)
g

Graph(num_nodes=3, num_edges=4,
      ndata_schemes={'h': Scheme(shape=(76,), dtype=torch.float32)}
      edata_schemes={'e': Scheme(shape=(14,), dtype=torch.float32)})

## DGLDatasetの作成
次に、DGLDatasetの作成をする。基本的なDGLDatasetの作成方法は``6_load_data.ipynb``を参照できるが、ここでは割愛する。今回は、[ZINC-250K](https://www.kaggle.com/datasets/basu369victor/zinc250k)のデータセットを使用する。簡単のため、今回は1000化合物のみを使用する。

In [7]:
# DataFrameの作成
df_molecule = pd.read_csv("250k_rndm_zinc_drugs_clean_3.csv")
df_molecule["smiles"] = df_molecule["smiles"].apply(lambda smi: smi.replace("\n", ""))
df_molecule = df_molecule.sample(n=1000, random_state=0) # 1000化合物のみ使用
df_molecule

,smiles,logP,qed,SAS
247668,CC(C)(C)OC(=O)N1CCC[C@H]1/C([O-])=N/S(C)(=O)=O,0.10430,0.544834,3.420845
180273,O=C(CC[NH2+][C@@H](c1ccc(F)cc1)C1CCCC1)N1CCCC1,2.63290,0.856875,3.391335
68403,CN(Cc1ccncc1)C(=O)CCNS(C)(=O)=O,-0.02070,0.811294,2.134847
48774,COc1ncnc(N2CCC[C@H]2C2CCCC2)c1N,2.22640,0.906907,2.968756
21866,C[C@H](NC(=O)NCC(C)(C)[NH+](C)C)c1ccc(C(F)(F)F...,1.98870,0.762620,3.469637
...,...,...,...,...
70310,COc1cc2nc([S-])n(Cc3ccc(N(C)C)cc3)c(=O)c2cc1OC,2.43380,0.507814,2.600695
233706,COc1ccc(N(C)S(=O)(=O)C2Cc3ccccc3C2)cc1OC,2.63720,0.831497,2.449949
59652,COc1cccc(CNC(=O)c2cccc(C)c2C)c1O,2.94764,0.899310,1.812122
204014,Cc1cc(-c2cc(F)cc3c2O[C@H](CNC(=O)c2ccco2)C3)ccc1F,4.26672,0.716957,2.896143


今回は、`MoleculeDataset`クラスを作成する。このクラスは、`DGLDataset`クラスを継承し、`__init__`メソッドでデータセットを読み込む。`process`メソッドでは、`__init__`の際に呼び出される、グラフセットとラベルを作成するための処理関数である。`__getitem__`メソッドはpytorchのDatasetと同じくインデックスを指定してデータを取得する。

In [8]:
class MoleculeDataset(DGLDataset):
    def __init__(self, df_molecule, target_columns=['logP'], num_process: int=8):
        self.df_molecule = df_molecule
        self.smiles = df_molecule["smiles"]
        self.labels = np.array(df_molecule[target_columns])
        self.num_process = num_process
        super().__init__(name="molecule")
    
    def mp_smiles_to_graph(self, args):
        smi = args[0]
        node_featurizer = args[1]
        edge_featurizer = args[2]
        g = smiles_to_bigraph(smi, 
            node_featurizer=mol_atom_featurizer,
            edge_featurizer=mol_bond_featurizer)
        return g
        
    def process(self):
        self.graphs = []
        # 並列化
        pool = multiprocessing.Pool(processes=self.num_process)
        
        args_list = [(smi, mol_atom_featurizer, mol_bond_featurizer) for smi in self.smiles]
        self.graphs = pool.map(self.mp_smiles_to_graph, args_list)
        pool.close()
        pool.join()
        
        self.labels = torch.tensor(self.labels, dtype=torch.float32)
    
    def __getitem__(self, i):
        return self.graphs[i], self.labels[i]

    def __len__(self):
        return len(self.graphs)

In [9]:
# データセットの作成
dataset = MoleculeDataset(df_molecule)
# 一つ目のデータを取り出す
graph, label = dataset[0]
print(graph, label)

Graph(num_nodes=19, num_edges=38,
      ndata_schemes={'h': Scheme(shape=(76,), dtype=torch.float32)}
      edata_schemes={'e': Scheme(shape=(14,), dtype=torch.float32)}) tensor([0.1043])


## データの分割

学習のために上記のデータセットを分割する。今回では、化合物データセットでよく用いられる方法である[`ScaffolfSplitter`](https://lifesci.dgl.ai/api/utils.splitters.html)を採用する。この方法は、化合物の骨格構造を基準にデータを分割する方法である。

他にもいくつか、分割手法が実装されている。詳細は[こちら](https://lifesci.dgl.ai/api/utils.splitters.html)を参照。
- ConsecutiveSplitter
- RandomSplitter
- MolecularWeightSplitter
- ScaffoldSplitter
- SingleTaskStratifiedSplitter

Splitterを使うときは、`k_fold_splitメソッド`あるいは`train_valid_test_split`メソッドを使用する。

In [10]:
from dgllife.utils import ScaffoldSplitter
splitter = ScaffoldSplitter()
train_dataset, valid_dataset, test_dataset = splitter.train_val_test_split(dataset, frac_val=0.1, frac_test=0.1)
print(len(train_dataset), len(valid_dataset), len(test_dataset))

## DataLoaderの作成
`GraphDataLoader`を利用して、DataLoaderを作成する。GraphDataLoaderはPytorchのDataLoaderを拡張したものであり、要領としてはPytorchのDataLoaderと同じである。

In [13]:
from dgl.dataloading import GraphDataLoader
train_loader = GraphDataLoader(
    dataset, batch_size=16, drop_last=False, shuffle=True
)

valid_loader = GraphDataLoader(
    dataset, batch_size=16, drop_last=False, shuffle=False
)

test_loader = GraphDataLoader(
    dataset, batch_size=16, drop_last=False, shuffle=False
)

## AttentiveFPの実装

次にGNNモデルの実装をする。今回は、dgllifeに実装されている`AttentiveFPGNN`モジュールを使用する。`AttentiveFPGNN`で出力されるテンソルはノードごとの特徴量であるので、これを集約するために、`dgl.mean_nodes(g, "h")`で特徴量の平均化を行う (cf. readout)。これにより、グラフ全体の特徴量を得ることができる。出力されたグラフ特徴量は、`MLP`レイヤに通して最終的な予測を行う。

In [14]:
from dgllife.model.gnn.attentivefp import AttentiveFPGNN
import torch
import torch.nn as nn
    
class AttentiveFP(nn.Module):
    def __init__(self, node_feat_size=76, edge_feat_size=14, 
                       num_layers=2, graph_feat_size=256, dropout=0.2, num_classes=1):
        super(AttentiveFP, self).__init__()
        self.attentive_fp = AttentiveFPGNN(
            node_feat_size=node_feat_size, edge_feat_size=edge_feat_size, num_layers=num_layers,
            graph_feat_size=graph_feat_size, dropout=dropout,
        )
        
        self.mlp = MLP(in_feats=256, h_feats=128, num_classes=1)
    
    def forward(self, g, node_feats, edge_feats):
        graph_feature = self.attentive_fp(g, node_feats, edge_feats)
        g.ndata["h"] = graph_feature
        mean_feature = dgl.mean_nodes(g, "h")
        out = self.mlp(mean_feature)
        return out

class MLP(nn.Module):
    def __init__(self, in_feats, h_feats, num_classes):
        super(MLP, self).__init__()
        self.lin1 = nn.Linear(in_feats, h_feats)
        self.lin2 = nn.Linear(h_feats, h_feats)
        self.out = nn.Linear(h_feats, num_classes)
        self.activation = nn.ReLU()

    def forward(self, x):
        x1 = self.activation(self.lin1(x))
        x2 = self.activation(self.lin2(x1) + x1)
        out = self.out(x2)
        return out

## 　学習の実行
最後に学習ループの実装を行い、実行する。dgllifeには性能評価のためのモジュールとして`Meter`クラスが実装されているので、今回はこれを使用する。
`Meter`オブジェクトは、`compute_metric(metric_name, reduction)`で指定したメトリックを計算することができる。詳細は[こちら](https://lifesci.dgl.ai/generated/dgllife.utils.Meter.html)を参照。

In [16]:
from tqdm import tqdm 
from dgllife.utils import Meter
import dgl
# Create the model with given dimensions

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

model = AttentiveFP(node_feat_size=76, edge_feat_size=14, 
                    num_layers=2, graph_feat_size=256, dropout=0.2, num_classes=1).to(device)

optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
criteria = nn.MSELoss()

train_meter, valid_meter, test_meter = Meter(), Meter(), Meter()

for epoch in range(50):
    model.train()
    print("="*50)
    print(f"Epoch: {epoch+1}")
    for batched_graph, labels in train_loader:
        batched_graph, labels = batched_graph.to(device), labels.to(device)
        pred = model(batched_graph, batched_graph.ndata["h"], batched_graph.edata['e'])
        loss = criteria(pred, labels)
        train_meter.update(pred, labels)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
    train_rmse = train_meter.compute_metric('rmse', reduction='mean')
    print(f"Train RMSE: {train_rmse}")    
    
    model.eval()
    for batched_graph, labels in valid_loader:
        batched_graph, labels = batched_graph.to(device), labels.to(device)
        pred = model(batched_graph, batched_graph.ndata["h"], batched_graph.edata['e'])
        valid_meter.update(pred, labels)
    valid_rmse = valid_meter.compute_metric('rmse', reduction='mean')
    print(f"Validation RMSE: {valid_rmse}")


model.eval()
for batched_graph, labels in test_loader:
    batched_graph, labels = batched_graph.to(device), labels.to(device)
    pred = model(batched_graph, batched_graph.ndata["h"], batched_graph.edata['e'])
    test_meter.update(pred, labels)
test_rmse = test_meter.compute_metric('rmse', reduction='mean')
print(f"Test RMSE: {test_rmse}")

Epoch: 1
Train RMSE: 1.5678720474243164
Validation RMSE: 1.2511006593704224
Epoch: 2
Train RMSE: 1.4290059804916382
Validation RMSE: 1.2390072345733643
Epoch: 3
Train RMSE: 1.3600316047668457
Validation RMSE: 1.1795523166656494
Epoch: 4
Train RMSE: 1.2824602127075195
Validation RMSE: 1.1023876667022705
Epoch: 5
Train RMSE: 1.200405478477478
Validation RMSE: 1.0481113195419312
Epoch: 6
Train RMSE: 1.1318678855895996
Validation RMSE: 0.995769202709198
Epoch: 7
Train RMSE: 1.0768791437149048
Validation RMSE: 0.947625994682312
Epoch: 8
Train RMSE: 1.0328803062438965
Validation RMSE: 0.9112719297409058
Epoch: 9
Train RMSE: 0.9929183721542358
Validation RMSE: 0.8764486312866211
Epoch: 10
Train RMSE: 0.9580897092819214
Validation RMSE: 0.8472912311553955
Epoch: 11
Train RMSE: 0.9278594255447388
Validation RMSE: 0.8214513063430786
Epoch: 12
Train RMSE: 0.9020091891288757
Validation RMSE: 0.7994392514228821
Epoch: 13
Train RMSE: 0.8779982328414917
Validation RMSE: 0.7807252407073975
Epoch: 14
T